In [1]:
# Import necessary libraries
import socket
import time
import threading

In [4]:
# Define the ServerMonitor class
class ServerMonitor(threading.Thread):
    def __init__(self, server_ip, server_port, check_interval=1):
        super().__init__()
        self.server_ip = server_ip
        self.server_port = server_port
        self.check_interval = check_interval
        self.running = True
        self.sock = None

    def reconnect(self):
        self.sock.close()  # Close the existing socket
        time_to_wait = 1  # Start with a 1-second wait
        max_wait = 32  # Maximum wait time of 32 seconds
        while self.running:
            try:
                print(f"Attempting to reconnect in {time_to_wait} seconds...")
                time.sleep(time_to_wait)
                self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                self.sock.connect((self.server_ip, self.server_port))
                self.sock.settimeout(3)
                print("Reconnected successfully.")
                break  # Exit the loop on successful reconnection
            except socket.error:
                print("Reconnection failed. Trying again...")
                time_to_wait = min(time_to_wait * 2, max_wait)

    def binary_states_to_hex_command(self, input_states):
        hex_digits = []
        for i in range(0, len(input_states), 4):
            binary_slice = input_states[i:i+4]
            binary_slice.reverse()
            hex_digit = hex(int(''.join(str(bit) for bit in binary_slice), 2))[2:].upper()
            hex_digits.append(hex_digit)
        hex_string = ''.join(hex_digits)
        command = f"@W04{hex_string}000000000000\r\n"
        return command

    def hex_to_binary(self, hex_value):
        return bin(int(hex_value, 16))[2:].zfill(4)

    def parse_io_states(self, response):
        hex_data = response[4:-2]
        binary_data = ''.join(self.hex_to_binary(hex_digit) for hex_digit in hex_data)
        input_states = ''.join(binary_data[i:i+4][::-1] for i in range(0, 16, 4))
        output_states = ''.join(binary_data[i:i+4][::-1] for i in range(16, 32, 4))
        input_states = [int(bit) for bit in input_states]
        output_states = [int(bit) for bit in output_states]
        print("Input states:", input_states)
        print("Output states:", output_states)
        return input_states, output_states

    def send_command(self, command):
        try:
            self.sock.send(command.encode('utf-8'))
            response = b''
            while True:
                part = self.sock.recv(1024)
                response += part
                if b'\r\n' in response:
                    break
            return response.decode('utf-8')
        except socket.timeout:
            print("Socket timeout occurred. Attempting to reconnect...")
            self.reconnect()
            return ""
        except OSError as e:
            print(f"An OSError occurred: {e}. Attempting to reconnect...")
            self.reconnect()
            return ""

    def run(self):
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.connect((self.server_ip, self.server_port))
            self.sock.settimeout(3)

            while self.running:
                read_command = '@R01\r\n'
                response = self.send_command(read_command)
                if response:
                    print("Server is up")
                input_states, output_states = self.parse_io_states(response)
                EtherCommand = self.binary_states_to_hex_command(input_states)
                ether_response = self.send_command(EtherCommand)
                time.sleep(self.check_interval)
        except Exception as e:
            print("An error occurred:", e)
        finally:
            if self.sock:
                self.sock.close()

    def stop(self):
        self.running = False

In [5]:
server_ip = '192.168.0.100'  # Replace with the actual server IP
server_port = 40001
monitor = ServerMonitor(server_ip, server_port)
monitor.start()

# Run the monitor for a specified duration then stop
time.sleep(10)  # Run for 10 seconds
monitor.stop()
monitor.join()

Server is up
Input states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Output states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Server is up
Input states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Output states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Server is up
Input states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Output states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Server is up
Input states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Output states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Server is up
Input states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Output states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Server is up
Input states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Output states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Server is up
Input states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Output states: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Server is up
Input s